# Демонстрация использования класса `SPEFAnalyzer`

Этот ноутбук показывает, как использовать класс `SPEFAnalyzer` из `analysis.spef` для обнаружения однофазных замыканий на землю (ОЗЗ) в осциллограммах.

Процесс включает:
- Загрузку осциллограмм с использованием `Oscillogram`.
- Нормализацию данных с помощью `OscillogramNormalizer`.
- Разделение данных по шинам/системам с помощью `OscillogramToCsvConverter` (используется его метод `split_buses`).
- Применение логики `SPEFAnalyzer` для детектирования ОЗЗ.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_spef_notebook_data/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json 
import csv # For creating sample norm.csv if not copying one
import sys

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from core.oscillogram import Oscillogram
from normalization.normalization import OscillogramNormalizer
from raw_to_csv.raw_to_csv import OscillogramToCsvConverter
from analysis.spef import SPEFAnalyzer
from analysis.overvoltage_detector import OvervoltageDetector # For copy_spef_oscillograms

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_for_spef_path = os.path.join(base_sample_data_dir, "comtrade_files", "for_spef_overvoltage_1")
config_samples_path = os.path.join(base_sample_data_dir, "config_files")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir_spef = "temp_spef_notebook_data"
notebook_source_dir_spef = os.path.join(notebook_base_temp_dir_spef, "source_oscillograms_spef")
notebook_output_dir_spef = os.path.join(notebook_base_temp_dir_spef, "spef_analysis_output")
notebook_config_dir_spef = os.path.join(notebook_base_temp_dir_spef, "config_files_spef")

if os.path.exists(notebook_base_temp_dir_spef):
    shutil.rmtree(notebook_base_temp_dir_spef)
os.makedirs(notebook_source_dir_spef)
os.makedirs(notebook_output_dir_spef)
os.makedirs(notebook_config_dir_spef)

# --- Копирование необходимых файлов для демонстрации ---
# Осциллограммы (spef_present.cfg/dat, no_spef.cfg/dat, spef_low_ov.cfg/dat (if used))
files_to_copy_cfgs = ["spef_present.cfg", "no_spef.cfg"] 
if os.path.exists(os.path.join(comtrade_samples_for_spef_path, "spef_symmetrical_phases.cfg")):
    files_to_copy_cfgs.append("spef_symmetrical_phases.cfg")

all_files_copied = True
for cfg_base_name in files_to_copy_cfgs:
    # Копируем CFG
    src_cfg_path = os.path.join(comtrade_samples_for_spef_path, cfg_base_name)
    if os.path.exists(src_cfg_path):
        shutil.copy(src_cfg_path, notebook_source_dir_spef)
    else:
        print(f"ОШИБКА: Исходный файл CFG не найден: {src_cfg_path}")
        all_files_copied = False
        continue # Пропускаем DAT если нет CFG
    
    # Копируем DAT
    dat_file_name = cfg_base_name[:-4] + ".dat"
    src_dat_path = os.path.join(comtrade_samples_for_spef_path, dat_file_name)
    if os.path.exists(src_dat_path):
        shutil.copy(src_dat_path, notebook_source_dir_spef)
    else:
        print(f"ОШИБКА: Исходный файл DAT не найден: {src_dat_path}")
        # Не устанавливаем all_files_copied в False, т.к. CFG может быть полезен для некоторых тестов
    
# Копирование/создание файлов конфигурации
src_norm_coef_path = os.path.join(config_samples_path, "norm_coeffs_for_ov_test.csv") 
dest_norm_coef_path_spef = os.path.join(notebook_config_dir_spef, "norm_coeffs_spef.csv")
if os.path.exists(src_norm_coef_path):
    shutil.copy(src_norm_coef_path, dest_norm_coef_path_spef)
else:
    print(f"ОШИБКА: Файл norm_coeffs_for_ov_test.csv не найден в {config_samples_path}.")
    all_files_copied = False

src_analog_names_path = os.path.join(config_samples_path, "dict_analog_names_sample.json")
dest_analog_names_path_spef = os.path.join(notebook_config_dir_spef, "dict_analog_names_spef.json")
if os.path.exists(src_analog_names_path): shutil.copy(src_analog_names_path, dest_analog_names_path_spef)
else: print(f"ОШИБКА: dict_analog_names_sample.json не найден."); all_files_copied = False
        
src_discrete_names_path = os.path.join(config_samples_path, "dict_discrete_names_sample.json")
dest_discrete_names_path_spef = os.path.join(notebook_config_dir_spef, "dict_discrete_names_spef.json")
if os.path.exists(src_discrete_names_path): shutil.copy(src_discrete_names_path, dest_discrete_names_path_spef)
else: print(f"ОШИБКА: dict_discrete_names_sample.json не найден."); all_files_copied = False

if not all_files_copied:
    print("Не все критичные файлы были скопированы. Демонстрация может быть неполной.")
else:
    print(f"Демонстрационные файлы скопированы в: {notebook_source_dir_spef} и {notebook_config_dir_spef}")
    print("Содержимое директории с осциллограммами:")
    for item in os.listdir(notebook_source_dir_spef): print(f"- {item}")

## 1. Инициализация компонентов

In [ ]:
spef_config = {
    'VALID_NOMINAL_VOLTAGES': {round(10000.0/np.sqrt(3), 2)}, # Phase RMS voltage in Volts
    'SPEF_THRESHOLD_U0': 0.06,  # PU (based on phase nominal RMS = 1 PU)
    'SPEF_THRESHOLD_Un': 0.05, # PU
    'SPEF_MIN_DURATION_PERIODS': 1, 
    'SIMILAR_AMPLITUDES_FILTER_ENABLED': True,
    'SIMILAR_AMPLITUDES_MAX_RELATIVE_DIFFERENCE': 0.15, 
    # 'verbose': True, # This would be handled by show_progress_bars in SPEFAnalyzer constructor
    'norm_yes_phrase': 'YES' 
}

normalizer = OscillogramNormalizer(norm_coef_file_path=dest_norm_coef_path_spef, is_print_message=True)

bus_splitter = OscillogramToCsvConverter(
    normalizer=normalizer, 
    raw_path=notebook_source_dir_spef, 
    csv_path=notebook_output_dir_spef, # csv_path is used by some methods if called, not essential for SPEFAnalyzer init
    dict_analog_names_path=dest_analog_names_path_spef,
    dict_discrete_names_path=dest_discrete_names_path_spef,
    is_print_message=True
)

norm_coeffs_df_loaded = pd.read_csv(dest_norm_coef_path_spef)
analyzer = SPEFAnalyzer(
    config=spef_config,
    normalizer=normalizer,
    bus_splitter=bus_splitter,
    norm_coef_df=norm_coeffs_df_loaded,
    show_progress_bars=True # Control tqdm progress bars
)
print("\nSPEFAnalyzer и зависимости инициализированы.")

## 2. Анализ директории на ОЗЗ (`analyze_directory`)

In [ ]:
output_spef_report_csv = os.path.join(notebook_output_dir_spef, "spef_report.csv")
error_log_spef_path = os.path.join(notebook_output_dir_spef, "spef_errors.log")

print(f"\nЗапуск анализа директории: {notebook_source_dir_spef}")
analyzer.analyze_directory(
    osc_folder_path=notebook_source_dir_spef,
    output_csv_path=output_spef_report_csv,
    log_path=error_log_spef_path
)

if os.path.exists(output_spef_report_csv):
    print(f"\nСодержимое отчета о ОЗЗ ({output_spef_report_csv}):")
    try:
        from IPython.display import display
        report_df = pd.read_csv(output_spef_report_csv)
        display(report_df)
    except ImportError:
        print(pd.read_csv(output_spef_report_csv))
    except Exception as e:
        print(f"Ошибка чтения CSV отчета: {e}")
else:
    print(f"Отчет о ОЗЗ не был создан.")

if os.path.exists(error_log_spef_path) and os.path.getsize(error_log_spef_path) > 0:
    print(f"\nСодержимое лога ошибок ({error_log_spef_path}):")
    with open(error_log_spef_path, "r", encoding='utf-8') as f: print(f.read())
else:
    print(f"\nЛог ошибок ({error_log_spef_path}) пуст или не создан.")

## 3. Демонстрация `detect_spef_on_bus_dataframe` (выборочно)

Покажем, как можно использовать внутренний метод `detect_spef_on_bus_dataframe` для анализа данных конкретной шины после их подготовки.

In [ ]:
try:
    osc_example_path = os.path.join(notebook_source_dir_spef, "spef_present.cfg") # Используем файл с ОЗЗ
    osc_example = Oscillogram(osc_example_path)
    print(f"Загружен {osc_example.file_hash} для детального анализа.")

    normalized_df_example = normalizer.normalize_bus_signals(
        osc_example.data_frame.copy(), 
        osc_example.file_hash,
        is_print_error=True
    )

    if normalized_df_example is not None:
        all_buses_example_df = bus_splitter.split_buses(
            normalized_df_example, # Normalizer returns df with time as column already
            os.path.basename(osc_example.filepath)
        )
        
        if all_buses_example_df is not None and not all_buses_example_df.empty:
            example_bus_group_name = all_buses_example_df['file_name'].unique()[0]
            example_bus_df = all_buses_example_df[all_buses_example_df['file_name'] == example_bus_group_name]
            
            print(f"Анализ группы шин: {example_bus_group_name}")
            samples_per_period_example = int(osc_example.cfg.sample_rates[0][0] / osc_example.frequency)
            
            is_spef = analyzer.detect_spef_on_bus_dataframe(
                example_bus_df, 
                samples_per_period_example,
                verbose_logging_method=True # Используем новый параметр, если класс обновлен
            )
            print(f"Результат detect_spef_on_bus_dataframe для {example_bus_group_name}: {is_spef}")
        else:
            print("Не удалось разделить данные по шинам для примера.")
    else:
        print("Не удалось нормализовать данные для примера.")
except Exception as e:
    print(f"Ошибка при детальном анализе: {e}")

## 4. Демонстрация `copy_spef_oscillograms`

In [ ]:
copied_spef_dir = os.path.join(notebook_output_dir_spef, "copied_spef_oscillograms")

if os.path.exists(output_spef_report_csv):
    print(f"\nКопирование осциллограмм из отчета {output_spef_report_csv} в {copied_spef_dir}...")
    # Используем OvervoltageDetector.copy_spef_oscillograms, т.к. у SPEFAnalyzer нет такого метода
    OvervoltageDetector.copy_spef_oscillograms(
        report_csv_path=output_spef_report_csv,
        source_osc_folder_path=notebook_source_dir_spef,
        destination_folder_path=copied_spef_dir,
        verbose_logging_method=True # Используем новый параметр, если класс обновлен
    )
    
    if os.path.exists(copied_spef_dir):
        print("\nСодержимое директории с скопированными ОЗЗ:")
        for item in os.listdir(copied_spef_dir):
            print(f"- {item}")
else:
    print(f"Отчет о ОЗЗ ({output_spef_report_csv}) не найден, копирование не будет выполнено.")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir_spef):
        shutil.rmtree(notebook_base_temp_dir_spef)
        print(f"\nВременная директория {notebook_base_temp_dir_spef} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir_spef} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir_spef}: {e}")